In [9]:
from transformers import BarkModel, AutoProcessor
from optimum.bettertransformer import BetterTransformer
from IPython.display import Audio
from scipy.io.wavfile import write as write_wav
import numpy as np
import torch


In [2]:
# set device
device = torch.device("cpu")   # Use CPU
if torch.cuda.is_available(): 
    device = torch.device("cuda")  # Use GPU
    n_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {n_gpus}")

print(f"Device: {device}")

Number of GPUs: 2
Device: cuda


In [11]:
model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16).to(device)
model = BetterTransformer.transform(model, keep_original_model=False)
processor = AutoProcessor.from_pretrained("suno/bark-small")
voice_preset = "v2/en_speaker_6"

/home/jaero/.miniforge3/envs/venv-carter/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The class `optimum.bettertransformers.transformation.BetterTransformer` is deprecated and will be removed in a future release.
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [12]:
# sample
input_text = "It sounds like you're on the right track with using a for loop to iterate through the elements and storing a cache of elements you've seen so far. [clears throat] Can you tell me a bit more about what you've tried so far and what's not working for you?"
inputs = processor(input_text, voice_preset=voice_preset)
output = model.generate(**inputs.to(device)).cpu().numpy().squeeze()

sample_rate = model.generation_config.sample_rate
print(type(output), output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<class 'numpy.ndarray'> [ 0.002636 -0.00224  -0.002056 ...  0.003128  0.002825  0.000329]


In [13]:
Audio(output, rate=sample_rate)

In [12]:
output = output.astype(np.float32)

In [13]:
write_wav("../data/sample_model.wav", sample_rate, output)

## Thoughts
- Works, but is kind of slow
- Voice is a bit inconsistent
- Will need to optimize for deployed state